In [30]:
import entity_tagging
import utils
import argparse
from datetime import datetime
import pandas as pd
import pickle
from sklearn.model_selection import KFold
import numpy as np

In [31]:
def load_corpus_xlsx(filename):
    corpus = pd.read_excel(filename, dtype={'msg': str})
    list_label = corpus['class'].to_list()
    list_content = corpus['msg'].to_list()

    return list_label, list_content

In [32]:
list_label, list_content = load_corpus_xlsx('./message_modified_v1.2.xlsx')

In [33]:
list_content = entity_tagging.entity_tagging(list_content)

In [34]:
classifier_list = ['nb', 'svm', 'dt', 'knn', 'maxent', 'baseline']

In [35]:
list_content

['vẫn như cũ á',
 'k bạn ơi mk đang dùng gói f16 bên bạn thấy bảo sắp chuyển sang gói f22 nên hỏi b ạ',
 'nhà anh dùng truền hình số fpt',
 'mình đang bận chắc cho kĩ thuật qua à',
 'đúng rồi em',
 'cố gắng giúp anh cảm ơn em',
 'vậy ở tỉnh thì khách có phải đặt cọc tiền modem đầu thu fpt tv khi có nhu cầu lắp không vậy bạn',
 'hiện tại bên em không phải là bộ phận tư vấn lắp mới nên không rõ về vấn đề trên ạ chính sách lắp đặt còn phụ thuộc vào từng chi nhánh',
 'mình có hỗ trợ đc ở đây ko hay phải gọi lên tổng đài ạ',
 'ngõ number doãn kế thiện mai dịch cầu giấy hà nội',
 'check giùm mình thông tin trên đc k',
 'truyen hinh sắp hết hạn đúng k b',
 'giờ cũng được',
 'em xin phép liên hệ anh ạ',
 'phạm hoàng ca number number number number date b nguyễn thị minh khai ninh kiều tp cần thơ',
 'ko còn cách nào ạ',
 'sao chưa có vậy không thì gửi qua gmail skirahoanganh gmail com',
 'có kịp xem đâu mà lần cuối',
 'mình muốn cắt không dùng tivi nữa thì làm như nào',
 'mạng càng ngày càng tệ 

# Begin test

In [10]:
#test_content = 'Giam gia 30-50% toan bo SP tu 13-16/11 tren toan he thong SR cua FPTshop nhan dip SN 9 nam'.lower()
#test_content_list = ['zxcvbnmasdfghjkl', 'Giam gia 30-50% toan bo SP tu 13-16/11 tren toan he thong SR cua FPTshop nhan dip SN 9 nam. Chi tiet tai www.fptshop.com.vn']

In [36]:
list_label, list_content = load_corpus_xlsx('./message_modified_v1.2.xlsx')

In [37]:
def test(list_content, test_content, list_label):
    content = list_content.copy()
    content.insert(0, test_content)
    content = entity_tagging.entity_tagging(content)
    # list_content_vec, list_len_sms, dictionary = utils.vectorize(content, 'bow')
    list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content[1:], list_label, 'bow')
    clf = utils.build_classifier_svm(list_content_vec, list_label)
    pickle.dump(clf, open(r'..\models\bow_svm_clf.pkl', 'wb'))
    
    # Reads in saved classification model
    load_clf = pickle.load(open(r'..\models\bow_svm_clf.pkl', 'rb'))

    prediction = load_clf.predict(list_content_vec[:1])

    return prediction

In [38]:
for content in test_content_list:
    prediction = test(list_content ,content, list_label)
    print(prediction)

ValueError: Found input variables with inconsistent numbers of samples: [6309, 6310]

In [39]:
list_content.insert(0, test_content)

In [40]:
list_content = entity_tagging.entity_tagging(list_content)

In [41]:
list_content_vec, list_len_sms, dictionary = utils.vectorize(list_content, 'tfidf')

In [42]:
list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content[1:], list_label, 'bow')

In [43]:
list_content_vec.shape

(6310, 2063)

In [44]:
clf = utils.build_classifier_svm(list_content_vec, list_label)
pickle.dump(clf, open(r'..\models\bow_svm_clf.pkl', 'wb'))

In [45]:
# Reads in saved classification model
load_clf = pickle.load(open(r'..\models\tfidf_svm_clf.pkl', 'rb'))

C:\Users\MINH.NV193012\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [46]:
list_content_vec[:1]

array([[1., 1., 1., ..., 0., 0., 4.]])

In [47]:
load_clf.predict(list_content_vec[:1])

array([0], dtype=int64)

# End test

In [48]:
def train_evaluation(list_label, list_content, vectorize_method, classifier_list):

    if vectorize_method == 'bow':
        list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content, list_label, 'bow')

    if vectorize_method == 'tfidf':
        list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content, list_label, 'tfidf')

    for i in classifier_list:
        list_false_positive, list_false_negative, list_true_positive, list_true_negative = \
        utils.kfold_classification(list_content, list_content_vec, list_label, i, 5, vectorize_method)
        # print(list_false_positive)
        # print(list_false_negative)
        # print(list_true_positive)
        # print(list_true_negative)
        print('Evaluation for ' + i.upper())
        utils.evaluation(list_false_positive, list_false_negative, list_true_positive, list_true_negative)
        print('-------------------------------------------------')

In [49]:
def train_entire_dataset(list_content, list_label, vectorize_method):
    list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content, list_label, vectorize_method)
    print(list_content_vec.shape)
    clf = utils.build_classifier_svm(list_content_vec, list_label)
    pickle.dump(clf, open(r'../\models\tfidf_svm_clf.pkl', 'wb'))
    print('Traning Done!')

In [50]:
train_entire_dataset(list_content, list_label,'tfidf')

(6311, 2064)


ValueError: Found input variables with inconsistent numbers of samples: [6311, 6310]

In [51]:
train_evaluation(list_label, list_content, 'bow', classifier_list)

Evaluation for NB
False Positive Rate: 0.9541800989593946%
False Negative Rate: 19.025578974670395%
True Positive Rate: 80.9744210253296%
True Negative Rate: 99.0458199010406%
-------------------------------------------------
Evaluation for SVM
False Positive Rate: 1.2223306058648087%
False Negative Rate: 21.840499634605436%
True Positive Rate: 78.15950036539456%
True Negative Rate: 98.77766939413519%
-------------------------------------------------
Evaluation for DT
False Positive Rate: 4.32384060630668%
False Negative Rate: 22.174555353699805%
True Positive Rate: 77.8254446463002%
True Negative Rate: 95.67615939369333%
-------------------------------------------------
Evaluation for KNN
False Positive Rate: 1.695334211614149%
False Negative Rate: 44.56268999812373%
True Positive Rate: 55.43731000187626%
True Negative Rate: 98.30466578838585%
-------------------------------------------------
Evaluation for MAXENT
False Positive Rate: 1.2221871235490043%
False Negative Rate: 22.324514

In [52]:
train_evaluation(list_label, list_content, 'tfidf', classifier_list)

Evaluation for NB
False Positive Rate: 0.05716447068948104%
False Negative Rate: 85.89971877561689%
True Positive Rate: 14.100281224383105%
True Negative Rate: 99.94283552931051%
-------------------------------------------------
Evaluation for SVM
False Positive Rate: 0.819530211314981%
False Negative Rate: 30.964333257453134%
True Positive Rate: 69.03566674254688%
True Negative Rate: 99.18046978868502%
-------------------------------------------------
Evaluation for DT
False Positive Rate: 5.315813996700265%
False Negative Rate: 25.574637953909605%
True Positive Rate: 74.4253620460904%
True Negative Rate: 94.68418600329971%
-------------------------------------------------
Evaluation for KNN
False Positive Rate: 2.3629499572274137%
False Negative Rate: 50.76606017370285%
True Positive Rate: 49.23393982629715%
True Negative Rate: 97.63705004277259%
-------------------------------------------------
Evaluation for MAXENT
False Positive Rate: 0.8387066104594467%
False Negative Rate: 33.23

     ---------------------------------------- 9.2/9.2 MB 8.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\MINH.NV193012\\AppData\\Local\\anaconda3\\Lib\\site-packages\\~-learn\\.libs\\msvcp140.dll'
Consider using the `--user` option or check the permissions.

